In [1]:
%pip install openai python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import logging

load_dotenv()

True

In [15]:
# Loading a single website
loader = WebBaseLoader("http://www.paulgraham.com/superlinear.html")

docs = loader.load()

# Split it up
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

# Embed and store your docs/vectors
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_EMBEDDING_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION"),
    azure_deployment="text-embedding-3-small",
)

vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [22]:
# Set logging for the queries
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Your original question
question = "What are the fundamental use cases of superlinear returns?"

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="gpt-4",
    temperature=0,
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the primary applications where superlinear returns are observed?', '2. In what scenarios do superlinear returns typically occur?', '3. Can you list examples of situations that lead to superlinear returns?']


In [21]:
chain = load_qa_chain(llm, chain_type="stuff")
answer = chain.invoke({"input_documents": unique_docs, "question":question})
print(answer['output_text'])

The fundamental use cases of superlinear returns, as mentioned in the context provided, reduce to two primary causes:

1. Exponential Growth: This is when you're working on something that grows exponentially, such as bacterial cultures. In such cases, the growth rate accelerates over time, leading to returns that are disproportionately large compared to the initial investment or effort.

2. Thresholds: This involves situations where there is a critical point that must be reached before any significant returns are realized. Once this threshold is crossed, the returns can increase dramatically, often more than proportionally to the additional effort or resources invested.

These two causes underpin many scenarios where superlinear returns are observed, from business and technology to social dynamics and natural phenomena.


In [23]:
prompt_template = """You are an AI language model assistant.

Your task is to generate 5 different versions of the given user question to retrieve relevant documents from a vector database.

By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations  of distance-based similarity search.

Provide these alternative questions separated by newlines.

Original question: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm, prompt=PROMPT
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. In what scenarios do superlinear returns typically occur?', '2. Can you list the primary applications where superlinear returns are observed?', '3. What are the key situations that lead to superlinear returns?', '4. Could you identify the basic instances where superlinear returns are significant?', '5. What are the core examples of phenomena that demonstrate superlinear returns?']
